# Multimodal Pattern-by-Pattern Demo with Metadata Inspection

This notebook demonstrates each of the 5 multimodal AI patterns individually, and shows the metadata tags stored in S3 Vector Store after each ingestion.

## The 5 Patterns:
1. **Text Pattern** - Process text-only descriptions
2. **Hybrid Pattern** - Combine text and images separately
3. **Full Embedding Pattern** - Unified multimodal understanding
4. **Describe Pattern** - Generate descriptions from images
5. **Summarize Pattern** - Condense long documents

After each pattern, we'll inspect the metadata tags stored in S3 Vector Store.

## Setup and Data Loading

In [ ]:
import os
import sys

# Run the setup script to configure imports
%run setup_imports.py

In [ ]:
# Additional imports for notebook
from typing import Dict, Any, List
import json
from IPython.display import Image, display, HTML
import matplotlib.pyplot as plt
from PIL import Image as PILImage
import boto3

print("✅ Notebook setup complete!")

In [ ]:
# Load sample data
def load_damage_data():
    """Load the multimodal damage data from JSON file."""
    data = load_json_data(MULTIMODAL_DAMAGE_DATA_FILE)
    return data['damage_cases']

def load_dealer_escalation_text():
    """Load the dealer escalation text for summarize pattern."""
    return load_text_data(DEALER_ESCALATION_FILE)

def get_image_path(damage_id):
    """Get the path to the damage image."""
    return os.path.join(IMAGES_DIR, f"{damage_id}.jpeg")

# Load the data
damage_cases = load_damage_data()
dealer_escalation_text = load_dealer_escalation_text()

print(f"📊 Loaded {len(damage_cases)} damage cases")
print(f"📄 Dealer escalation text: {len(dealer_escalation_text):,} characters")

In [ ]:
# Create MMIngestor and S3 client
def create_mm_ingestor():
    """Create MMIngestor with image resizing for automotive photos."""
    vector_bucket_name, object_bucket_name, index_name = get_standard_names()
    
    mm_ingestor = MMIngestor(
        index_name=index_name,
        region_name=REGION_NAME
    )
    
    # Configure image resizer for automotive damage photos
    mm_ingestor.preprocessor_chain.clear()
    image_resizer = ImageResizer(
        target_size=(512, 384),
        preserve_aspect_ratio=True
    )
    mm_ingestor.add_preprocessor(image_resizer)
    
    return mm_ingestor

def inspect_document_metadata(doc_id, pattern_name):
    """Query and display metadata for a document."""
    if not doc_id:
        print("❌ No document ID available for metadata query")
        return
        
    print(f"🔍 Querying S3 Vector Store for {pattern_name} metadata...")
    
    # Get S3 Vector Store details
    vector_bucket_name, object_bucket_name, index_name = get_standard_names()
    index_name = index_name[index_name.find("/") +1 :]
    s3_vectors_client = boto3.client('s3vectors', region_name=REGION_NAME)
    
    try:
        # Get vector details
        response = s3_vectors_client.get_vectors(
            vectorBucketName=vector_bucket_name,
            indexName=index_name,
            keys=[doc_id],
            returnMetadata=True
        )
        
        if response.get('vectors'):
            vector_data = response['vectors'][0]
            metadata = vector_data.get('metadata', {})
            
            print(f"📊 {pattern_name.upper()} PATTERN - Stored Metadata Tags:")
            print("=" * 60)
            print(f"📄 Document ID: {doc_id}")
            print(f"🏷️  Total Metadata Tags: {len(metadata)}")
            print()
            
            # Display metadata in organized way
            if any(key in metadata for key in ['vehicle_make', 'vehicle_model', 'vehicle_year']):
                print("🚗 Vehicle Information:")
                for key in ['vehicle_make', 'vehicle_model', 'vehicle_year']:
                    if key in metadata:
                        print(f"   {key}: {metadata[key]}")
                print()
            
            if any(key in metadata for key in ['damage_type', 'severity', 'estimated_cost']):
                print("💥 Damage Information:")
                for key in ['damage_type', 'severity', 'estimated_cost']:
                    if key in metadata:
                        print(f"   {key}: {metadata[key]}")
                print()
            
            if any(key in metadata for key in ['document_type', 'category', 'dealer', 'customer']):
                print("📄 Document Information:")
                for key in ['document_type', 'category', 'dealer', 'customer', 'case_id']:
                    if key in metadata:
                        print(f"   {key}: {metadata[key]}")
                print()
            
            print("🔧 Processing Information:")
            for key in ['pattern', 'pattern_demo']:
                if key in metadata:
                    print(f"   {key}: {metadata[key]}")
            print()
            
            # Pattern-specific metadata
            if 'text_length' in metadata or 'text_word_count' in metadata:
                print("📝 Text Processing Details:")
                for key in ['text_length', 'text_word_count', 'embedding_dimension', 'text_preview']:
                    if key in metadata:
                        value = metadata[key]
                        if key == 'text_preview' and len(str(value)) > 100:
                            value = str(value)[:100] + '...'
                        print(f"   {key}: {value}")
                print()
            
            if 'has_image' in metadata or 'image_size' in metadata:
                print("🖼️  Image Processing Details:")
                for key in ['has_image', 'image_size', 'image_mode', 'image_format', 'image_stored']:
                    if key in metadata:
                        print(f"   {key}: {metadata[key]}")
                print()
            
            if 'summary' in metadata or 'summary_length' in metadata:
                print("📋 Summary Details:")
                for key in ['summary_length', 'original_text_length', 'compression_ratio']:
                    if key in metadata:
                        print(f"   {key}: {metadata[key]}")
                if 'summary' in metadata:
                    summary = metadata['summary']
                    if len(summary) > 200:
                        summary = summary[:200] + '...'
                    print(f"   summary_preview: {summary}")
                print()
            
            if 'description' in metadata:
                print("🤖 AI-Generated Description:")
                description = metadata['description']
                if len(description) > 200:
                    description = description[:200] + '...'
                print(f"   description_preview: {description}")
                if 'description_length' in metadata:
                    print(f"   description_length: {metadata['description_length']}")
                print()
            
            print("🏷️  All Metadata Tags:")
            for key, value in sorted(metadata.items()):
                if isinstance(value, str) and len(value) > 50:
                    display_value = value[:50] + '...'
                else:
                    display_value = value
                print(f"   {key}: {display_value}")
        else:
            print("❌ No vector data found")
            
    except Exception as e:
        print(f"❌ Error querying metadata: {e}")

# Create the ingestor
mm_ingestor = create_mm_ingestor()
print(f"🔧 MMIngestor created with patterns: {mm_ingestor.pattern_engine.list_patterns()}")

---
# Pattern 1: TEXT PATTERN

**What it does**: Processes only written damage descriptions using standard text embeddings

**When to use**: When you have detailed written damage reports but no images

**Example**: Insurance adjuster writes detailed damage assessment

In [ ]:
# Select a case for text pattern demonstration
text_case = damage_cases[0]  # Honda Civic case

print("🚗 TEXT PATTERN DEMO")
print("=" * 50)
print(f"Vehicle: {text_case['metadata']['vehicle_year']} {text_case['metadata']['vehicle_make']} {text_case['metadata']['vehicle_model']}")
print(f"Damage Type: {text_case['metadata']['damage_type'].replace('_', ' ').title()}")
print(f"Estimated Cost: ${text_case['metadata']['estimated_cost']}")
print(f"Severity: {text_case['metadata']['severity'].title()}")
print()
print("📝 Text Description:")
print(f"\"{text_case['damage_text']}\"")
print()
print("🔍 What the TEXT pattern will do:")
print("• Convert text to numerical embeddings using sentence-transformers")
print("• Store metadata for filtering and search")
print("• Enable semantic search based on text content only")

In [ ]:
# Ingest using TEXT pattern
print("🔄 Processing with TEXT pattern...")

metadata = {
    'damage_id': text_case['id'],
    'vehicle_make': text_case['metadata']['vehicle_make'],
    'vehicle_model': text_case['metadata']['vehicle_model'],
    'vehicle_year': text_case['metadata']['vehicle_year'],
    'damage_type': text_case['metadata']['damage_type'],
    'severity': text_case['metadata']['severity'],
    'estimated_cost': text_case['metadata']['estimated_cost'],
    'pattern_demo': 'text_pattern'
}

try:
    text_pattern_doc_id = mm_ingestor.ingest(
        content={'text': text_case['damage_text']},
        metadata=metadata,
        pattern="text"
    )
    print(f"✅ Successfully processed with TEXT pattern")
    print(f"📄 Document ID: {text_pattern_doc_id}")
    print(f"🎯 Pattern: text (text-only processing)")
    print(f"📊 Input: {len(text_case['damage_text'])} characters of text")
    
except Exception as e:
    print(f"❌ Error: {e}")
    text_pattern_doc_id = None

In [ ]:
# Inspect TEXT pattern metadata
inspect_document_metadata(text_pattern_doc_id, "TEXT")

---
# Pattern 2: HYBRID PATTERN

**What it does**: Combines photos with text. Generates embeddings for text and stores photo to object store and adds meta-data tag

**When to use**: When you have both damage photos and written assessments

**Example**: Insurance claim with both photos and adjuster notes

In [ ]:
# Select a case for hybrid pattern demonstration
hybrid_case = damage_cases[1]  # Toyota Camry case
hybrid_image_path = get_image_path(hybrid_case['id'])

print("🚗📸 HYBRID PATTERN DEMO")
print("=" * 50)
print(f"Vehicle: {hybrid_case['metadata']['vehicle_year']} {hybrid_case['metadata']['vehicle_make']} {hybrid_case['metadata']['vehicle_model']}")
print(f"Damage Type: {hybrid_case['metadata']['damage_type'].replace('_', ' ').title()}")
print(f"Estimated Cost: ${hybrid_case['metadata']['estimated_cost']}")
print()
print("📝 Text Description:")
print(f"\"{hybrid_case['damage_text']}\"")
print()
print("📸 Image Description:")
print(f"\"{hybrid_case['image_description']}\"")
print()
print("🔍 What the HYBRID pattern will do:")
print("• Process text and image separately")
print("• Create separate embeddings for text and visual content")
print("• Combine both for comprehensive search capabilities")
print("• Enable search across both text and visual information")

In [ ]:
# Display the image
try:
    img = PILImage.open(hybrid_image_path)
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title(f"Damage Photo: {hybrid_case['id']}")
    plt.axis('off')
    plt.show()
    print(f"📸 Image loaded: {hybrid_image_path}")
except Exception as e:
    print(f"❌ Could not load image: {e}")

In [ ]:
# Ingest using HYBRID pattern
print("🔄 Processing with HYBRID pattern...")

metadata = {
    'damage_id': hybrid_case['id'],
    'vehicle_make': hybrid_case['metadata']['vehicle_make'],
    'vehicle_model': hybrid_case['metadata']['vehicle_model'],
    'vehicle_year': hybrid_case['metadata']['vehicle_year'],
    'damage_type': hybrid_case['metadata']['damage_type'],
    'severity': hybrid_case['metadata']['severity'],
    'estimated_cost': hybrid_case['metadata']['estimated_cost'],
    'pattern_demo': 'hybrid_pattern'
}

try:
    hybrid_pattern_doc_id = mm_ingestor.ingest(
        content={'text': hybrid_case['damage_text'],
                 'image': hybrid_image_path},
        metadata=metadata,
        pattern="hybrid"
    )
    print(f"✅ Successfully processed with HYBRID pattern")
    print(f"📄 Document ID: {hybrid_pattern_doc_id}")
    print(f"🎯 Pattern: hybrid (text + image separately)")
    print(f"📊 Input: {len(hybrid_case['damage_text'])} characters of text + image")
    print(f"🖼️  Image: {hybrid_image_path}")
except Exception as e:
    print(f"❌ Error: {e}")
    hybrid_pattern_doc_id = None

In [ ]:
# Inspect HYBRID pattern metadata
inspect_document_metadata(hybrid_pattern_doc_id, "HYBRID")

---
# Pattern 3: FULL EMBEDDING PATTERN

**What it does**: Creates unified embeddings from both text and images for comprehensive understanding

**When to use**: For the best cross-modal search experience where text and images are equally important

**Example**: Complex damage assessment requiring both visual and textual analysis

In [ ]:
# Select a case for full embedding pattern demonstration
full_case = damage_cases[2]  # BMW X5 case (severe damage)
full_image_path = get_image_path(full_case['id'])

print("🚗🧠 FULL EMBEDDING PATTERN DEMO")
print("=" * 50)
print(f"Vehicle: {full_case['metadata']['vehicle_year']} {full_case['metadata']['vehicle_make']} {full_case['metadata']['vehicle_model']}")
print(f"Damage Type: {full_case['metadata']['damage_type'].replace('_', ' ').title()}")
print(f"Estimated Cost: ${full_case['metadata']['estimated_cost']}")
print(f"Severity: {full_case['metadata']['severity'].title()}")
print()
print("📝 Text Description:")
print(f"\"{full_case['damage_text']}\"")
print()
print("📸 Image Description:")
print(f"\"{full_case['image_description']}\"")
print()
print("🔍 What the FULL EMBEDDING pattern will do:")
print("• Create a single unified representation from both text and image")
print("• Use advanced multimodal AI to understand relationships")
print("• Enable the most sophisticated cross-modal search")
print("• Best for complex damage requiring both visual and textual understanding")

In [ ]:
# Display the image
try:
    img = PILImage.open(full_image_path)
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title(f"Severe Damage Photo: {full_case['id']}")
    plt.axis('off')
    plt.show()
    print(f"📸 Image loaded: {full_image_path}")
except Exception as e:
    print(f"❌ Could not load image: {e}")

In [ ]:
# Ingest using FULL EMBEDDING pattern
print("🔄 Processing with FULL EMBEDDING pattern...")

metadata = {
    'damage_id': full_case['id'],
    'vehicle_make': full_case['metadata']['vehicle_make'],
    'vehicle_model': full_case['metadata']['vehicle_model'],
    'vehicle_year': full_case['metadata']['vehicle_year'],
    'damage_type': full_case['metadata']['damage_type'],
    'severity': full_case['metadata']['severity'],
    'estimated_cost': full_case['metadata']['estimated_cost'],
    'pattern_demo': 'full_embedding_pattern'
}

try:
    full_pattern_doc_id = mm_ingestor.ingest(
        content={'text': full_case['damage_text'],
                 'image': full_image_path},
        metadata=metadata,
        pattern="full_embedding"
    )
    print(f"✅ Successfully processed with FULL EMBEDDING pattern")
    print(f"📄 Document ID: {full_pattern_doc_id}")
    print(f"🎯 Pattern: full_embedding (unified multimodal understanding)")
    print(f"📊 Input: {len(full_case['damage_text'])} characters of text + image")
    print(f"🖼️  Image: {full_image_path}")
    print(f"🧠 Result: Single unified embedding representing both text and visual content")
except Exception as e:
    print(f"❌ Error: {e}")
    full_pattern_doc_id = None

In [ ]:
# Inspect FULL EMBEDDING pattern metadata
inspect_document_metadata(full_pattern_doc_id, "FULL EMBEDDING")

---
# Pattern 4: DESCRIBE PATTERN

**What it does**: Generates text descriptions from images, then processes as text

**When to use**: When you have images but limited or no text descriptions

**Example**: Photos from mobile app submissions where users didn't provide detailed descriptions

In [ ]:
# Select a case for describe pattern demonstration
describe_case = damage_cases[3]  # Ford Explorer case
describe_image_path = get_image_path(describe_case['id'])

print("📸➡️📝 DESCRIBE PATTERN DEMO")
print("=" * 50)
print(f"Vehicle: {describe_case['metadata']['vehicle_year']} {describe_case['metadata']['vehicle_make']} {describe_case['metadata']['vehicle_model']}")
print(f"Damage Type: {describe_case['metadata']['damage_type'].replace('_', ' ').title()}")
print(f"Estimated Cost: ${describe_case['metadata']['estimated_cost']}")
print()
print("📸 We have this image:")
print(f"Image description: \"{describe_case['image_description']}\"")
print()
print("📝 Original text (we'll use minimal text for this demo):")
print(f"\"{describe_case['damage_text']}\"")
print()
print("🔍 What the DESCRIBE pattern will do:")
print("• Analyze the image using computer vision")
print("• Generate a detailed text description of what it sees")
print("• Process the generated description as text")
print("• Perfect for when you have photos but minimal text")

In [ ]:
# Display the image
try:
    img = PILImage.open(describe_image_path)
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title(f"Image for AI Description: {describe_case['id']}")
    plt.axis('off')
    plt.show()
    print(f"📸 Image loaded: {describe_image_path}")
    print("🤖 The AI will analyze this image and generate a description...")
except Exception as e:
    print(f"❌ Could not load image: {e}")

In [ ]:
# Ingest using DESCRIBE pattern
print("🔄 Processing with DESCRIBE pattern...")
print("⏳ This may take a moment as AI analyzes the image...")

metadata = {
    'damage_id': describe_case['id'],
    'vehicle_make': describe_case['metadata']['vehicle_make'],
    'vehicle_model': describe_case['metadata']['vehicle_model'],
    'vehicle_year': describe_case['metadata']['vehicle_year'],
    'damage_type': describe_case['metadata']['damage_type'],
    'severity': describe_case['metadata']['severity'],
    'estimated_cost': describe_case['metadata']['estimated_cost'],
    'pattern_demo': 'describe_pattern'
}

try:
    describe_pattern_doc_id = mm_ingestor.ingest(
        content={'text': describe_case['damage_text'],  # Minimal text
                 'image': describe_image_path},
        metadata=metadata,
        pattern="describe"
    )
    print(f"✅ Successfully processed with DESCRIBE pattern")
    print(f"📄 Document ID: {describe_pattern_doc_id}")
    print(f"🎯 Pattern: describe (AI-generated description from image)")
    print(f"📊 Input: Image + minimal text")
    print(f"🖼️  Image: {describe_image_path}")
    print(f"🤖 AI generated a detailed description from the image")
except Exception as e:
    print(f"❌ Error: {e}")
    describe_pattern_doc_id = None

In [ ]:
# Inspect DESCRIBE pattern metadata
inspect_document_metadata(describe_pattern_doc_id, "DESCRIBE")

---
# Pattern 5: SUMMARIZE PATTERN

**What it does**: Condenses lengthy reports into key points before processing

**When to use**: For long documents, case files, or detailed documentation

**Example**: Multi-page dealer escalation cases, detailed inspection reports

In [ ]:
print("📄➡️📋 SUMMARIZE PATTERN DEMO")
print("=" * 50)
print("📄 Document Type: Dealer Escalation Case")
print(f"📊 Original Length: {len(dealer_escalation_text):,} characters")
print(f"📝 Word Count: ~{len(dealer_escalation_text.split()):,} words")
print()
print("📖 Document Preview (first 500 characters):")
print(f"\"{dealer_escalation_text[:500]}...\"")
print()
print("🔍 What the SUMMARIZE pattern will do:")
print("• Analyze the entire long document")
print("• Extract key information and main points")
print("• Create a concise summary focusing on important details")
print("• Process the summary for efficient search and retrieval")
print("• Perfect for lengthy technical reports, case files, and documentation")

In [ ]:
# Show the full document in a scrollable area
display(HTML(f"""
<div style="height: 300px; overflow-y: scroll; border: 1px solid #ccc; padding: 10px; background-color: #f9f9f9;">
<h4>📄 Full Dealer Escalation Document:</h4>
<pre style="white-space: pre-wrap; font-size: 12px;">{dealer_escalation_text}</pre>
</div>
"""))

print("☝️ Scroll through the document above to see the full content that will be summarized.")

In [ ]:
# Ingest using SUMMARIZE pattern
print("🔄 Processing with SUMMARIZE pattern...")
print("⏳ This may take a moment as AI analyzes and summarizes the document...")

metadata = {
    'document_type': 'dealer_escalation',
    'category': 'customer_service',
    'dealer': 'Lakeside Honda',
    'customer': 'Mrs. Janet T.',
    'vehicle': '2021 Honda Civic EX',
    'case_id': '2024-0934',
    'pattern_demo': 'summarize_pattern'
}

try:
    summarize_pattern_doc_id = mm_ingestor.ingest(
        content={'text': dealer_escalation_text},
        metadata=metadata,
        pattern="summarize"
    )
    print(f"✅ Successfully processed with SUMMARIZE pattern")
    print(f"📄 Document ID: {summarize_pattern_doc_id}")
    print(f"🎯 Pattern: summarize (AI-generated summary of long document)")
    print(f"📊 Input: {len(dealer_escalation_text):,} characters")
    print(f"📋 AI created a concise summary focusing on key information")
    print(f"🔍 The summary is now searchable and can be quickly retrieved")
except Exception as e:
    print(f"❌ Error: {e}")
    summarize_pattern_doc_id = None

In [ ]:
# Inspect SUMMARIZE pattern metadata
inspect_document_metadata(summarize_pattern_doc_id, "SUMMARIZE")

---
# Search Demonstration

Now let's search across all the patterns we've ingested to see how they perform:

In [ ]:
def print_search_results(results: List[Dict[str, Any]], query: str):
    """Print search results in a clean format with pattern information."""
    print(f"\n🔍 Query: '{query}'")
    print(f"Found {len(results)} results:")
    print("=" * 60)
    
    for i, result in enumerate(results, 1):
        metadata = result.get('metadata', {})
        pattern = metadata.get('pattern', 'unknown')
        score = result['similarity_score']
        
        print(f"\n{i}. Score: {score:.3f} | Pattern: {pattern.upper()}")
        
        if 'vehicle_make' in metadata and 'vehicle_model' in metadata:
            vehicle = f"{metadata.get('vehicle_year', '')} {metadata['vehicle_make']} {metadata['vehicle_model']}"
            damage_type = metadata.get('damage_type', '').replace('_', ' ').title()
            cost = metadata.get('estimated_cost', 'N/A')
            print(f"   🚗 Vehicle: {vehicle}")
            print(f"   💥 Damage: {damage_type}")
            print(f"   💰 Cost: ${cost}")
        
        if 'document_type' in metadata:
            doc_type = metadata['document_type'].replace('_', ' ').title()
            print(f"   📄 Document: {doc_type}")
            if 'dealer' in metadata:
                print(f"   🏢 Dealer: {metadata['dealer']}")
            if 'case_id' in metadata:
                print(f"   🔢 Case ID: {metadata['case_id']}")
        
        # Show which demo pattern this came from
        if 'pattern_demo' in metadata:
            demo_pattern = metadata['pattern_demo'].replace('_', ' ').title()
            print(f"   🎯 Demo Pattern: {demo_pattern}")
        
        print("-" * 40)

In [ ]:
# Search 1: Look for bumper damage (should find text pattern)
query1 = "bumper damage Honda"
results1 = mm_ingestor.search(query={'text': query1}, top_k=3)
print_search_results(results1, query1)

In [ ]:
# Search 2: Look for severe collision (should find full embedding pattern)
query2 = "severe collision damage BMW"
results2 = mm_ingestor.search(query={'text': query2}, top_k=3)
print_search_results(results2, query2)

In [ ]:
# Search 3: Look for dealer escalation (should find summarize pattern)
query3 = "Honda dealer customer service issue"
results3 = mm_ingestor.search(query={'text': query3}, top_k=3)
print_search_results(results3, query3)

In [ ]:
# Search 4: General damage search (should find multiple patterns)
query4 = "vehicle damage assessment"
results4 = mm_ingestor.search(query={'text': query4}, top_k=5)
print_search_results(results4, query4)

---
# Pattern Comparison Summary

## Metadata Tags by Pattern:

### 🔤 TEXT Pattern Metadata:
- **Core tags**: `pattern`, `text_length`, `text_word_count`, `embedding_dimension`
- **Preview**: `text_preview` (first part of processed text)
- **Custom**: Your business metadata (vehicle info, damage details)

### 🔤📸 HYBRID Pattern Metadata:
- **Text tags**: Same as TEXT pattern
- **Image tags**: `has_image`, `image_size`, `image_mode`, `image_stored`
- **Processing**: `processed_at`, separate embeddings for text and image

### 🧠 FULL EMBEDDING Pattern Metadata:
- **Multimodal**: `multimodal`, `has_text`, `has_image`
- **Unified**: Single embedding representing both text and image
- **Advanced**: `image_processed`, `image_format` details

### 📸➡️📝 DESCRIBE Pattern Metadata:
- **AI Description**: `description`, `description_length`
- **Original**: `original_text`, `original_text_length`
- **Processing**: `processing_type` = 'describe'

### 📄➡️📋 SUMMARIZE Pattern Metadata:
- **Summary**: `summary`, `summary_length`
- **Compression**: `compression_ratio`, `original_text_length`
- **Efficiency**: Condensed content for faster search

## Key Insights:
- Each pattern creates **different metadata tags** optimized for its use case
- **Filterable tags** enable precise search and categorization
- **Processing details** help understand what AI operations were performed
- **Business metadata** remains consistent across all patterns for filtering